In [1]:
import pandas as pd
import scipy as sc
import numpy as np
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt
import seaborn as sb


In [2]:
knesset20eclections = pd.read_excel(r"Knesset Elections\-20-.xlsx")
knesset20eclections = knesset20eclections.drop("מספר קלפי", axis=1)
knesset20eclections = knesset20eclections.groupby(by=["שם ישוב","סמל ישוב" ], axis=0).sum()

knesset19eclections = pd.read_excel(r"Knesset Elections\-19-.xlsx")
knesset19eclections = knesset19eclections.drop("מספר קלפי", axis=1)
knesset19eclections = knesset19eclections.groupby(by=["שם ישוב","סמל ישוב" ], axis=0).sum()

knesset18eclections = pd.read_excel(r"Knesset Elections\-18-.xlsx")
knesset18eclections = knesset18eclections.drop("סמל קלפי", axis=1)
knesset18eclections = knesset18eclections.groupby(by=["שם ישוב","סמל ישוב" ], axis=0).sum()

knesset17eclections = pd.read_excel(r"Knesset Elections\-17-2006.xls")
knesset17eclections = knesset17eclections.drop("מספר קלפי", axis=1)
knesset17eclections = knesset17eclections.groupby(by=["שם ישוב","סמל ישוב" ], axis=0).sum()


knesset16eclections = pd.read_excel(r"Knesset Elections\-16-2003.xls")
knesset16eclections = knesset16eclections.drop("סמל קלפי", axis=1)
knesset16eclections = knesset16eclections.groupby(by=["שם ישוב","סמל ישוב" ], axis=0).sum()


knesset15eclections = pd.read_excel(r"Knesset Elections\-15-1999-.xls")
knesset15eclections = knesset15eclections.drop("מס' רץ", axis=1).drop("קלפי", axis=1)
knesset15eclections = knesset15eclections.groupby(by=["שם ישוב","סמל ישוב" ], axis=0).sum()


knesset14eclections = pd.read_excel(r"Knesset Elections\-14-1996-.xls")
knesset14eclections = knesset14eclections.drop("סמל קלפי", axis=1).drop("כתובת", axis=1)
knesset14eclections = knesset14eclections.groupby(by=["שם ישוב","סמל ישוב" ], axis=0).sum()



In [4]:
knesset17eclections.head()
knesset18eclections.head()
knesset19eclections.head()
knesset14eclections.head()

,,פיצול,"ס""ה בוחרים","ס""ה מצביעים","ס""ה כשרים","ס""ה פסולים",אמת,ב,ג,ד,דן,...,מרץ,נ,ע,פ,פא,ק,שס,ת,Unnamed: 30,Unnamed: 31
שם ישוב,סמל ישוב,,,,,,,,,,,,,,,,,,,,,
אבו ג'ווייעד,967,3.0,886,728,711,17,57,0,2,0,1,...,0,1,629,4,5,0,2,0,0,0
אבו גוש,472,3.0,2243,1552,1463,89,378,23,20,1,1,...,227,3,559,0,6,0,24,0,0,0
אבו סנאן,473,3.0,4925,3961,3824,137,749,324,6,3,1,...,275,2,596,5,33,5,42,2,0,0
אבו עבדון,958,0.0,164,93,89,4,7,0,0,0,0,...,14,0,50,1,0,0,3,0,0,0
אבו עמאר,1042,0.0,126,108,102,6,11,0,0,0,0,...,0,0,64,0,24,0,0,0,0,0


In [5]:

psulim_percentage = knesset14eclections['ס"ה מצביעים']/knesset14eclections['ס"ה בוחרים']
psulim_percentage.sort_values(ascending=False)

שם ישוב            סמל ישוב
משמר איילון        670         1.105839
נצרים              5408        1.085714
ורד יריחו          3639        1.029703
כפר דרום           5405        1.011765
גן נר              1274        1.007890
אבני חפץ           3793        0.991736
חגי                3764        0.988506
אלמוג              3556        0.982143
יסודות             440         0.975728
כרמי צור           3766        0.975410
ח'ואלד             986         0.972332
כלנית              1229        0.970149
מגדל עוז           3561        0.967213
דולב               3747        0.960784
שקף                1233        0.956044
בית אל ב'          3714        0.951515
סוסיה              3756        0.950980
כפר טרומן          673         0.947917
איתמר              3762        0.946903
עוקבי )בנו עוקבה(  957         0.946058
טלמון              3788        0.946043
לפידות             1173        0.944444
אעבלין             529         0.943904
כפר קאסם           634         0.941522
מתתיהו      

,,מצביעים,פסולים,כשרים,אמת,ג,ד,הי,הפ,הק,ו,...,פך,פנ,פץ,ץ,צפ,צק,ק,קנ,רק,שס
שם ישוב,סמל ישוב,,,,,,,,,,,,,,,,,,,,,
אבו ג'ווייעד (שבט,967,217,11,206,2,0,13,0,0,0,0,...,0,0,0,0,1,0,0,0,0,4
אבו גוש,472,1844,79,1765,118,7,644,0,2,0,99,...,0,3,3,7,31,4,4,4,1,106
אבו סנאן,473,4732,65,4667,221,3,372,1,5,1,1594,...,0,2,4,7,292,2,15,8,6,366
אבו עבדון (שבט),958,69,0,69,0,0,7,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
אבו קורינאת (שבט),968,660,10,650,5,0,45,0,0,0,10,...,0,2,2,0,2,1,6,3,5,79


In [85]:
psulim_percentage = knesset19eclections["פסולים"]/knesset19eclections["מצביעים"]
psulim_percentage.sort_values(ascending=False)
print(psulim_percentage["תל אביב - יפו"])
print(psulim_percentage["ירושלים"])

סמל ישוב
5000    0.008056
dtype: float64
סמל ישוב
3000    0.007814
dtype: float64
